# sentiment dictionary

In [16]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [109]:
def tf_extractor(corpus):  
    # returns a frequency-based DTM
    vectorizer = CountVectorizer(min_df=1, ngram_range=(1,1)) 
    features = vectorizer.fit_transform(corpus) # transform texts to a frequency matrix
    return vectorizer, features  

In [110]:
def tfidf_extractor(corpus):
    # returns a tf-idf based DTM
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

## <br>
## naver_clova

In [112]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_naver_clova.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(2556, 5)

In [113]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 2 -> 부정, -1
    # 4 ~ 5 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [114]:
# You can also use the following method
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)

In [115]:
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
# input의 형태 = list of docs
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]
# tf_vectorizer.vocabulary_.items() returns a list of (word, frequency)
# We sort words based on their frequencies and save the words

In [119]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
# C = Inverse of regularization strength, 즉 C 값이 작을수록 penalty를 많이 준다는 것입니다.
# penalty를 많이 준다는 뜻은 L1 같은 경우는 feature의 수를 그만큼 많이 줄인다는 뜻이고
# L2인 경우는 weight 값을 더 0에 가깝게 한다는 뜻입니다.
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 178 out of 641
Accuracy: 0.72


In [38]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 304 out of 641
Accuracy: 0.53


In [41]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(7574, 1.039721417408199), (7600, 0.7724308659319985), (7561, 0.5665838283947136), (1049, 0.47337489103995595), (7547, 0.4468903456873619)]
좋아요 (1.040)
좋은데 (0.772)
좋네요 (0.567)
근데 (0.473)
좋겠어요 (0.447)
좋겠네요 (0.432)
좋겠습니다 (0.384)
같아요 (0.351)
있게 (0.320)
좋습니다 (0.318)
부르면 (0.315)
감사합니다 (0.311)
ㅎㅎ (0.302)
빅스비 (0.284)
있었으면 (0.278)
추가해주세요 (0.273)
하지만 (0.267)
합니다 (0.260)
해주세요 (0.252)
되면 (0.249)
있으면 (0.238)
good (0.231)
정말 (0.230)
빅스비보다 (0.219)
다좋은데 (0.218)
너무좋아요 (0.210)
클로바를 (0.209)
저는 (0.208)
시리처럼 (0.205)
좋고 (0.204)
좋은 (0.203)
재생이 (0.203)
더욱 (0.198)
좋음 (0.196)
주세요 (0.185)
좋을것 (0.184)
한가지 (0.183)
최고 (0.178)
있는데요 (0.173)
굿굿 (0.167)
연동 (0.159)
같습니다 (0.157)
하네요 (0.156)
아주 (0.155)
에서 (0.152)
쓰고 (0.151)
음성호출이 (0.148)
인공지능 (0.147)
이름이 (0.146)
좋겠다 (0.145)
샀는데 (-0.190)
도대체 (-0.194)
앱을 (-0.197)
음성 (-0.197)
같은데 (-0.198)
어플도 (-0.199)
스피커도 (-0.205)
혼자 (-0.207)
갑자기 (-0.208)
나오네요 (-0.209)
점점 (-0.209)
그냥 (-0.210)
없음 (-0.219)
해도 (-0.221)
안되는게 (-0.221)
어떻게 (-0.225)
너무 (-0.229)
프렌즈 (-0.229)
클로바 (-0.233)
않는 (-0.2

In [64]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\naver_clova.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## kakao_mini

In [65]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_kakao_mini.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(578, 5)

In [66]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [67]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [71]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 50 out of 149
Accuracy: 0.66


In [69]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 69 out of 149
Accuracy: 0.54


In [72]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(2523, 0.607919781186865), (2512, 0.3326869482487577), (2511, 0.25866577529010176), (2708, 0.24190978331699442), (2516, 0.18162850739750763)]
좋아요 (0.608)
좋겠어요 (0.333)
좋겠습니다 (0.259)
카카오미니 (0.242)
좋네요 (0.182)
기능이 (0.180)
좋고 (0.164)
음악 (0.159)
좋은데 (0.158)
있으면 (0.146)
업데이트 (0.145)
제가 (0.142)
ㅠㅠ (0.117)
기대합니다 (0.114)
근데 (0.112)
있습니다 (0.110)
노래도 (0.107)
잘쓰고 (0.106)
너무 (0.105)
유용하게 (0.105)
위젯이 (0.104)
부탁드려요 (0.104)
좋다 (0.102)
헤이카카오 (0.102)
지금은 (0.101)
해주세요 (0.098)
있어서 (0.097)
원하는 (0.096)
좋습니다 (0.095)
아쉽네요 (0.091)
제발 (0.090)
영어로 (0.089)
그게 (0.087)
베이스 (0.087)
음질도 (0.083)
신기하고 (0.080)
다른건 (0.080)
좋은데요 (0.080)
같아요 (0.079)
기능 (0.078)
앞으로 (0.078)
반응도 (0.076)
만족합니다 (0.075)
있었으면 (0.075)
하지 (0.074)
설정이 (0.074)
재생할때 (0.072)
빠른 (0.072)
다만 (0.072)
최고 (0.072)
미니 (-0.088)
사용 (-0.088)
같은 (-0.088)
와이파이도 (-0.089)
안잡히고 (-0.091)
이게 (-0.094)
잡음이 (-0.095)
불러도 (-0.095)
와이파이를 (-0.097)
없다고 (-0.099)
해결해주세요 (-0.102)
연결도 (-0.103)
공유기 (-0.104)
안되는데 (-0.104)
카카오 (-0.105)
있어요 (-0.105)
무슨 (-0.109)
불편해요 (-0.112)
바로 (-0.11

In [73]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\kakao_mini.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## kt_gigagenie

In [74]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_kt_gigagenie.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(430, 5)

In [75]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [76]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [79]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 32 out of 110
Accuracy: 0.71


In [78]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 55 out of 110
Accuracy: 0.50


In [80]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(1142, 0.6547985427460136), (1224, 0.28839288216950926), (1151, 0.21039604604230958), (1282, 0.13418928234346778), (460, 0.11387123016243433)]
좋아요 (0.655)
최고 (0.288)
좋음 (0.210)
편해요 (0.134)
매일 (0.114)
kt간편로그인에 (0.112)
좋다 (0.092)
좋네요 (0.091)
good (0.087)
좋은 (0.084)
듭니다 (0.083)
아쉽네요 (0.082)
간편하고 (0.079)
제가 (0.071)
아주 (0.064)
우리집 (0.063)
근데 (0.059)
등록된 (0.056)
아이디가 (0.056)
아이디를 (0.056)
좋습니다 (0.048)
취향 (0.048)
너무감사쥬쥬 (0.048)
말귀를잘알아들음 (0.048)
기가지니가티비아니면핸드폰에나오는데난기가지니를처음써본다 (0.048)
김정순 (0.048)
좋아좋아 (0.048)
넘좋아요ㅎㅎ (0.048)
교통상황빨리 (0.048)
편함 (0.048)
멋져뿌러 (0.048)
조찬복 (0.048)
재재가재재를먹어부렸어 (0.048)
좋아ㅡ (0.048)
네좋아요 (0.048)
다좋음 (0.048)
넘냐ㅡㅎ은것 (0.048)
좋을것갔다 (0.048)
좋아여 (0.048)
지니를궃이않불러도돼서편리해요 (0.048)
조아 (0.048)
헤헤 (0.048)
오케 (0.048)
감사 (0.048)
간단하네ㅎㅎ (0.048)
디ㅡㄱㅅㄱㅅ (0.048)
편리해요 (0.048)
ㅠㅠ5555 (0.048)
기가지니는참말을잘듣습니다 (0.048)
않되는데 (0.048)
쓰고 (-0.077)
굉장히 (-0.077)
지금 (-0.078)
담거나 (-0.079)
해주세요 (-0.079)
노래가 (-0.081)
정말 (-0.081)
권한 (-0.082)
장난까냐 (-0.082)
기능이 (-0.082)
늘려주세요 (-0.083)
ㅡㅡ (-0.083)
갑자기 (-0.083)
쓸데

In [81]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\kt_gigagenie.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## skt_nugu

In [82]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data = pd.read_csv(path + r'\playstore_review_skt_nugu.csv', encoding='UTF-8')
comments = list(data['comment'])
ratings = list(data['ratings'])
data.shape

(1940, 5)

In [83]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [84]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [87]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=0.1, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 134 out of 488
Accuracy: 0.73


In [86]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C=0.1, penalty='l1', solver='saga') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 175 out of 488
Accuracy: 0.64


In [88]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(5314, 1.1386396414224107), (5308, 0.4173584755355908), (4866, 0.33210715718373546), (5311, 0.32803802155508194), (3434, 0.31923048287435984)]
좋아요 (1.139)
좋네요 (0.417)
있습니다 (0.332)
좋습니다 (0.328)
아리아 (0.319)
좋겠습니다 (0.307)
good (0.286)
좋겠어요 (0.276)
최고 (0.268)
기능이 (0.260)
좋음 (0.241)
감사합니다 (0.233)
있으면 (0.220)
좋은 (0.202)
좋은데 (0.199)
편해요 (0.195)
많이 (0.193)
사용하고 (0.175)
기대됩니다 (0.162)
기능은 (0.161)
생각보다 (0.148)
nugu (0.144)
너무좋아요 (0.144)
정말 (0.142)
랜덤 (0.141)
아주 (0.135)
있어요 (0.135)
기능들이 (0.133)
혹시 (0.129)
추가해주세요 (0.128)
있어서 (0.125)
제가 (0.122)
편리해요 (0.120)
있도록 (0.119)
아주좋아요 (0.118)
좋아 (0.117)
다좋은데 (0.116)
있었으면 (0.115)
난리 (0.115)
잘쓰고 (0.114)
최고의 (0.113)
있는데 (0.112)
크게 (0.112)
조아요 (0.111)
편하고 (0.111)
좋겠네요 (0.111)
있는 (0.109)
좀더 (0.107)
보입니다 (0.107)
유튜브 (0.106)
연결을 (-0.166)
끊겨서 (-0.167)
처럼 (-0.168)
사용 (-0.171)
안되요 (-0.171)
제대로 (-0.171)
접속이 (-0.171)
앱은 (-0.174)
멜론을 (-0.177)
진짜 (-0.178)
무조건 (-0.185)
아무리 (-0.185)
연결할수 (-0.186)
인터넷 (-0.188)
멜론 (-0.188)
없네요 (-0.188)
업데이트 (-0.188)
누르면 (-0.190)
말을 (-0.193)
나

In [89]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\skt_nugu.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

## <br>
## clova + nugu + gigagenie + mini

In [127]:
path = r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data'
data1 = pd.read_csv(path + r'\playstore_review_naver_clova.csv', encoding='UTF-8')
data2 = pd.read_csv(path + r'\playstore_review_skt_nugu.csv', encoding='UTF-8')
data3 = pd.read_csv(path + r'\playstore_review_kt_gigagenie.csv', encoding='UTF-8')
data4 = pd.read_csv(path + r'\playstore_review_kakao_mini.csv', encoding='UTF-8')
comments = list(data1['comment']) + list(data2['comment']) + list(data3['comment']) + list(data4['comment'])
ratings = list(data1['ratings']) + list(data2['ratings']) + list(data3['ratings']) + list(data4['ratings']) 

In [56]:
len(comments)

5504

In [128]:
filtered_texts = []
filtered_labels = []

for text, score in zip(comments, ratings):
    if score == 3:
        continue
        
    filtered_texts.append(text)
    filtered_labels.append(1 if score > 3 else -1)

In [129]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)
tf_vectorizer, train_tf_features = tf_extractor(train_texts)
test_tf_features = tf_vectorizer.transform(test_texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [104]:
tf_vectorizer.vocabulary_

{'저는': 2770,
 '멜론을': 1054,
 '사용해서': 1503,
 '못하네요': 1134,
 '멜론도': 1047,
 '추가해주시면': 3106,
 '좋을것같아요': 2921,
 '수준으로': 1635,
 '오류투성이': 2301,
 '특히': 3214,
 '음악재생시': 2440,
 '지멋대로': 2989,
 '아님': 1805,
 '클로바': 3162,
 '기기를': 353,
 '샀고': 1525,
 '해보니': 3421,
 '다좋은데': 647,
 '이거': 2453,
 '하나만': 3321,
 '추가해주세요': 3105,
 '폰에': 3272,
 '다운로드한': 638,
 '음악': 2426,
 '구글': 273,
 '플레이': 3295,
 '뮤직도': 1201,
 '네이버뮤직': 528,
 '연동': 2248,
 '어떻게하나요': 2052,
 '하는거': 3328,
 '보고': 1316,
 'ㄷㄷ': 75,
 '있어서': 2650,
 '가까운': 90,
 '가는': 95,
 '자동차': 2674,
 '알려드릴게요': 1979,
 '셀리야': 1609,
 '조아요': 2864,
 '인식률과': 2567,
 '성능이': 1600,
 '좋아요': 2904,
 '인식': 2562,
 '불가하네요': 1382,
 'good': 38,
 '그럭저럭': 316,
 '쓰고': 1745,
 '있습니다': 2647,
 '많이': 973,
 '질문': 3012,
 '다시': 628,
 '실행도': 1704,
 '필요합니다': 3310,
 '해서': 3425,
 '사용해': 1502,
 '음성인식이': 2420,
 '수준입니다': 1638,
 '전에': 2793,
 '우리집': 2353,
 'ㅋㅋ': 78,
 '정말': 2820,
 '있었으면': 2654,
 '그리고': 334,
 '발음을': 1267,
 '알아': 1989,
 '들으면': 818,
 '좋을거': 2919,
 '같아요': 139,
 '앱이': 2028,
 '업뎃': 2132,
 '하면': 335

### Test accuracy <br>
- naver_clova : 0.72 (641) <br>
- skt_nugu : 0.73 (488) <br>
- kt_gigagenie : 0.71 (110) <br>
- kakao_mini : 0.66 (149) <br>

In [123]:
# C : 0.1 -> 1 변경시 accuracy 증가

In [131]:
# tf matrix를 사용한 경우
lr = LogisticRegression(C=2, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tf_features, train_labels) # 학습
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 298 out of 1387
Accuracy: 0.79


In [118]:
# tfidf matrix를 사용한 경우
tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
lr = LogisticRegression(C= 3, penalty='l2', solver='sag') # Lasso regression
lr.fit(train_tfidf_features, train_labels) # 학습
pred_labels = lr.predict(test_tfidf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))

Misclassified samples: 276 out of 1387
Accuracy: 0.80


In [119]:
# Get coefficients of the model
coefficients = lr.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

print(sorted_coefficients[:5])
# print top 50 positive words
for word, coef in sorted_coefficients[:50]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))
# print top 50 negative words
for word, coef in sorted_coefficients[-50:]:
    print('{0:} ({1:.3f})'.format(vocablist[word], coef))

[(13457, 5.125966722215377), (13502, 4.7814764715665055), (13439, 3.3672700042443915), (13450, 3.1796278555224253), (13525, 2.9910970849798493)]
좋아요 (5.126)
좋은데 (4.781)
좋네요 (3.367)
좋습니다 (3.180)
좋음 (2.991)
최고 (2.928)
good (2.919)
감사합니다 (2.795)
근데 (2.770)
좋겠어요 (2.767)
너무좋아요 (2.572)
있습니다 (2.427)
추가해주세요 (2.402)
같아요 (2.329)
제가 (2.194)
좋아 (2.192)
아주좋아요 (2.187)
좋은 (2.186)
좋겠습니다 (2.171)
아리아 (2.142)
좋겠네요 (2.134)
다좋은데 (2.078)
있었으면 (2.026)
편해요 (2.023)
좋고 (1.999)
감사 (1.973)
굿굿 (1.919)
최고의 (1.890)
좋다 (1.872)
좋네 (1.836)
잘쓰고 (1.835)
있으면 (1.833)
빅스비보다 (1.792)
최고에요 (1.784)
쵝오 (1.769)
조아요 (1.717)
다만 (1.629)
가끔 (1.586)
신기하네요 (1.572)
그래도 (1.537)
그런데 (1.524)
되면 (1.523)
짱짱 (1.514)
합니다 (1.500)
필요해요 (1.495)
nugu (1.468)
유용한 (1.441)
없어도 (1.433)
하지만 (1.428)
고마워요 (1.427)
모르겠어요 (-1.390)
오류가 (-1.401)
음성 (-1.437)
샀는데 (-1.443)
티비 (-1.456)
비해 (-1.463)
네트워크 (-1.491)
절대 (-1.493)
아니고 (-1.496)
쓰레기 (-1.510)
블루투스 (-1.545)
말을 (-1.549)
않는 (-1.579)
차라리 (-1.584)
못하고 (-1.588)
어플이 (-1.609)
못알아 (-1.663)
제발 (-1.678)
없고 (-1.689)
안되

In [120]:
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\senti_dictionary.txt', 'w', encoding='utf-8') as f:
    for word, coef in sorted_coefficients:
        f.write(vocablist[word]+'\t'+str(coef)+'\n')
f.close()

In [121]:
import re 
sent_dict = {}
with open(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\sentiment_dictionary\senti_dictionary.txt', 'r', encoding='utf-8') as f:
    for i in f.readlines():
        word, score = i.split('\t')
        score = re.sub('\n', '', score)
        sent_dict[word] = float(score)
f.close()

In [122]:
sent_dict

{'좋아요': 5.125966722215377,
 '좋은데': 4.7814764715665055,
 '좋네요': 3.3672700042443915,
 '좋습니다': 3.1796278555224253,
 '좋음': 2.9910970849798493,
 '최고': 2.9280054694703055,
 'good': 2.918856920427324,
 '감사합니다': 2.7952949036076906,
 '근데': 2.769657472434236,
 '좋겠어요': 2.7667791573891676,
 '너무좋아요': 2.5716149226563063,
 '있습니다': 2.427431480392143,
 '추가해주세요': 2.4015228951305634,
 '같아요': 2.3285969633091836,
 '제가': 2.194446779498704,
 '좋아': 2.1917368055366033,
 '아주좋아요': 2.1865503825819075,
 '좋은': 2.1861348198190673,
 '좋겠습니다': 2.171456241359408,
 '아리아': 2.1419339755926536,
 '좋겠네요': 2.1341052674313197,
 '다좋은데': 2.077607361509783,
 '있었으면': 2.0261018149603243,
 '편해요': 2.0232555594899653,
 '좋고': 1.999100191869628,
 '감사': 1.972989897183361,
 '굿굿': 1.9188733903596948,
 '최고의': 1.8904034198436246,
 '좋다': 1.872224701017496,
 '좋네': 1.8360810230099718,
 '잘쓰고': 1.8352729486287167,
 '있으면': 1.8327857097106577,
 '빅스비보다': 1.7924537121175075,
 '최고에요': 1.7839882118445327,
 '쵝오': 1.7693882603462685,
 '조아요': 1.71681114158

In [179]:
sent_dict = {}
for word, coef in sorted_coefficients:
    sent_dict[vocablist[word]] = coef
sent_dict

{'좋아요': 2.9526365397046006,
 '좋은데': 2.2516683380580385,
 'good': 1.8489078844881726,
 '좋네요': 1.8166668418466116,
 '좋음': 1.7243950817841833,
 '좋습니다': 1.6710564911966266,
 '최고': 1.6198631536759365,
 '감사합니다': 1.5744289983787203,
 '너무좋아요': 1.4445207357646836,
 '근데': 1.3259092550497764,
 '있습니다': 1.3068808073905174,
 '아주좋아요': 1.2344609762629262,
 '좋아': 1.1826321553619878,
 '추가해주세요': 1.1788512496661605,
 '다좋은데': 1.1552033651041658,
 '편해요': 1.078104430783355,
 '좋고': 1.0731711592158804,
 '감사': 1.0642715017158606,
 '같아요': 1.045057518697363,
 '좋은': 1.0376158331857264,
 '굿굿': 1.0017470375943998,
 '제가': 0.9891542349881555,
 '최고의': 0.9869545265957445,
 '좋다': 0.9791651319408108,
 '좋네': 0.9564719608267904,
 '잘쓰고': 0.9311742073965216,
 '최고에요': 0.9128656680345015,
 '쵝오': 0.9054240777772419,
 '좋겠네요': 0.9050567615154744,
 '아리아': 0.9013321454268016,
 '그런데': 0.8910966324881633,
 'nugu': 0.8882264637177401,
 '조아요': 0.8870544501824127,
 '빅스비보다': 0.8821219668651791,
 '좋겠어요': 0.8725736657432833,
 '있었으면': 0.8653

# <br>
# wordcloud

## 감성사전으로 긍정, 부정, 중립 문헌 나누기

In [7]:
df = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\naver_blog\naverclova_speaker_blog.csv', encoding='UTF-16')
texts = list(df['text'].dropna())
texts = list(map(lambda x: x.replace('\n', ' ').replace('\r', ' '), texts))

### RogisticRegression 이용하여 나누기

### 문장 별로 나누었을 때

In [8]:
import nltk.tokenize
sentences = list(map(nltk.tokenize.sent_tokenize, texts))

In [15]:
lr_pos, lr_neg =  [],[]
for sentence in sentences:
    lr_pos_sen, lr_neg_sen = [], []
    test_tf_features = tf_vectorizer.transform(sentence)
#    print(test_tf_features)
    pred_labels = lr.predict(test_tf_features)
    
    for num, score in enumerate(pred_labels):
        if score == 1:
            lr_pos_sen.append(sentence[num])
        else:
            lr_neg_sen.append(sentence[num])
    lr_pos.append(lr_pos_sen)
    lr_neg.append(lr_neg_sen)

In [16]:
print(len(lr_pos)), print(len(lr_neg))

3405
3405


(None, None)

In [118]:
lr_pos

[['침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두고 저녁 생활을 도울 괜찮은 기능의 블루투스 스피커 + 라디오 알람시계를 소개합니다.',
  '블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) 모닝콜 2, 이 제품은 책상이나 주방, 특히 침대맡에 두기 딱 좋은 다기능 라디오 알람시계입니다.',
  '딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진에서 보이는 것보다 좀 작습니다.',
  '한 뼘 조금 안되는 아담 사이즈.',
  '침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요.',
  '밤에 조명을 끈 방에서는 LED시계가 필요하죠.',
  '그런데 시계의 화면이 엄청 큰!!',
  '!아주 잘 보여요.',
  '^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요.',
  '제 아내처럼.',
  '그런 분들은 LED 밝기를 5단계로 조절하시면 됩니다.',
  '(스누즈 버튼) ▼ \xa0● Radio : 정해진 시간에 라디오가 켜집니다\xa0● Buzzer : 설정시간에 뚜뚜 부저음이 울려요아침에 두 번의 알람을 설정해 울릴 수 있고 두 가지 종류 중 선택합니다.',
  '아침에는 두 개를 시간차 설정해서 확실히 깨면 되겠죠?',
  '라디오로 깰 분위기 잡고 부저 알람으로 발딱 일어나기.',
  '(혹은 거꾸로...) ㅎㅎ 요게 라디오 알람시계를 찾는 이유 아니겠어요?※ 스누즈 버튼을 누르면 약 9분 후 알람이 다시 울립니다 주파수를 탐색해 듣거나 6개 채널 주파수가 저장이 됩니다.',
  'Sleep버튼으로 일정 시간 후 자동으로 꺼지도록 설정할 수 있어요.',
  '(15, 30, 45, 60, 90분) ※ 안테나는 긴 "선"이 뒤에 달려 있습니다.',
  '펼쳐서 벽에 붙이거나 적절히 위치를 조절하면 신호가 잘 잡혀요.',
  '스마트폰과 블루투스 연결로 MP3나 애플뮤직, 벅스 팟캐스트 같은 서비스의 스트

In [70]:
pd.DataFrame(sum(lr_pos,[]))

,0
0,"침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두..."
1,블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) ...
2,딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진...
3,한 뼘 조금 안되는 아담 사이즈.
4,"침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요."
5,밤에 조명을 끈 방에서는 LED시계가 필요하죠.
6,그런데 시계의 화면이 엄청 큰!!
7,!아주 잘 보여요.
8,^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요.
9,제 아내처럼.


In [74]:
pd.DataFrame(sum(lr_neg,[]))
# for t in pd.DataFrame(sum(lr_neg,[]))[0]:
#     print(t)

,0
0,ㅋㅋ<체크사항> 반드시 전원을 연결해야 작동을 하는 제품입니다.
1,"어학부터 책, 트렌드, 정보 등 음성 위주의 들을꺼리가 가득!"
2,♬ 교양이 솔솔~( 다운로드 : 애플 앱스토어 / 구글 플레이 ) ■ Mixer B...
3,MP3 준비 없이도 쓱~♬ (앱 설명 포스팅 보기)Mixerbox 3 ( 애플 앱스...
4,WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 ...
5,간신히 와이파이 설정 단계를 넘어 간 이후 AI 스피커 활용사례로 제일 많이 꼽히는...
6,"""MY > 이용권 구매 > 내 이용권 정보"" 페이지의 맨 하단에 있다....ㅠㅠ정말..."
7,구글 홈구글의 인공지능 서비스를 바탕으로 다양한 서비스를 진행 중입니다.자세한 내용...
8,애플 홈팟올해 12월에 판매가 될 예상으로 이미 다양한 정보들이 다 공개가 되었습니...
9,이미 상용화되고 출시도 되어 있고한국어에 100% 호환성이 있기 때문에 사용할 때는...


### 11번가 review로 performance 확인

In [132]:
review1 = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\review_data_nugu.csv')
review2 = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\review_data_clova.csv')
review3 = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\review_data_kakao.csv')
review4 = pd.read_csv(r'C:\Users\sormd\Documents\GitHub\ai_speaker_textmining\data\review_data_gigagenie.csv')

review1 = review1.drop(review1[review1.nugu.isna()].index)
review2 = review2.drop(review2[review2.nugu.isna()].index)
review3 = review3.drop(review3[review3.nugu.isna()].index)
review4 = review4.drop(review4[review4.nugu.isna()].index)

scores = list(review1.score) + list(review2.score) + list(review3.score) + list(review4.score)
comments = list(review1.nugu) + list(review2.nugu) + list(review3.nugu) + list(review4.nugu)

In [133]:
review_texts = []
review_labels = []

for text, score in zip(comments, scores):
    if score == 3:
        continue
        
    review_texts.append(text)
    review_labels.append(1 if score > 3 else -1)

In [96]:
for text, score in zip(nugu, scores):
    if score <3:
        print(text)

여기서 사는거 서비스센터갔더니 정품이아니고 등록도 안되어있다고 AS안됩니다. 그래서 통신사 고객센터랑 네이버 고객센터 다 물어보고 했는데도 AS센터에서 말하는 등록하는게 없다고합니다. 그냥 고장나면 끝입니다. 1회용이에요. 절대 비추입니다.
인터넷에서사면 유플러스 티비랑 연종이안되네요ㅜㅜ
톡으로 상담후 배송요청란에 기재후 주문했는데, 깡그리무시. 배송당일날 전화로 반품해준다고서, 다음날 또 불가하다고 문자옴.상담은 왜하고, 배송요청란에 적으라는얘긴왜했는지.
이거 엘지유플러스에 등록하려고하는데 자급제라고 등록이안된다고합니다. 이거정품이 아닌건가요? 등록을 해야지만 as수리가 가능하다는데 그럼 수리도 못받는 제품인가요?
배송 중 분실했다느니 지연됐다느니 말도 많고 탈도 많았네요
판매에는 문제가없지만 클로바 자체가 문제가 많내요.언어전달도 잘안되고 사용언어 인식도 너무 수준이 낮아요,10살딸이 쓴다고 용돈모아샀는데 돈만원 짜리면 ..그것도 아까운 제품이내요
살짝!!음성인식이늦어요
음질이 최악이네여 반품 귀찮 다시 사야할듯
일단 물건 자체는 좋습니다.하지만 새 물건이 아닌건 확실한 거 같아요.클로바 바닥도 지저분하고 뜯고보니 속눈썹같은 털과 하트스티커같은게 붙어있는데 이게 뭔가 싶네요.여기 반품 상품 제대로 검수안하고 재판매하나요?답변 제대로 해주시길 바랍니다.물건 자체는 만족스럽지만 생일이라서 샀는데 기분이 더럽네요.미니언즈는 .
모든 리모컨 호환된다더니..ㅜ.ㅜ안되요.저희집 TV는 없네요.그래서 등록이 안되요.미니언즈랑 폰이랑 연결했는데 왜 미니언즈 눈을 눌러야 명령을 듣는건지..폰 클로버 누르면 폰따로 미니언즈 따로..괜히 비싼돈 날렸어요.
18년 5월 제조한 물건을 보내셨네요~19년 3월인데 ㅠ 재고처리도 아니고아주 기분 나쁩니다.
생각보다 기능이나 인식율이 별로네요


In [134]:
test_tf_features = tf_vectorizer.transform(review_texts)
pred_labels = lr.predict(test_tf_features)
print('Misclassified samples: {} out of {}'.format((pred_labels != review_labels).sum(), len(review_labels)))
print('Accuracy: %.2f' % accuracy_score(review_labels, pred_labels))

Misclassified samples: 1097 out of 4355
Accuracy: 0.75


In [107]:
count = 0
for num, i in enumerate(review_labels):
    if i == -1:
        count += 1
print(count)

936


In [108]:
936/4355

0.21492537313432836

In [104]:
for num, i in enumerate(review_labels):
    if i == -1:
        print(review_texts[num])

귀도 어둡고, 머리도 나쁘고, 그저 노래만 부를 줄 아는군
솔직한 사용기 입니다. 일단 음성인식 블루투스 스피커 + 무드등을하나 산다고 해도 이가격이면 괜찮다고 생각합니다. 음성 인식률이나 다양한 정보는 아직 한참 미흡합니다. 정해진 규칙에 어느정도 맞춰서 대화해야하고 그외에는 다 알아듣지 못한다고 보면 됩니다. 음악하나는 잘 틀어줍니다. 그리고 이번에 업데이트되서 무드등기능이 좀더 업그레이드 되어 보기 좋더군요. 앞으로 소스도 공개하고 응용범위도 확대해 나간다니 기대해볼랍니다. SK니깐 손놓고 있진 않겠지요. 아마존의 에코, 구글의 구글홈 이제품들과 비교해서 일단 메리트는 한국말 대답이자연스럽고 멜론과 연동되는 정도 그리고 음질은 안꿀린다...이정도 인것 같습니다..아직은.... 초등학교 들어가기전 애들 언어교육할 상대 정도만 돼도 참 좋겠더군요. 시리정도 기대하시면 부족합니다...대신 목소리는 더좋아요..자연스럽고 이상 간단 솔직구매후기 였습니다.
말귀를 정말 못알아들음 ^^ 스피커랑 무드등 샀다고 생각할게요!
음성 인식 인공지능이라고 해서 구매했는 데 음성인식으로 되는 것은 음악 들려주는 것 밖에는 없고 볼륨 조절도 음성 인식으로 안되고 스마트폰으로 연동되어야만 메뉴나 설정이 변경되는 어처구니 없는 인공지능. 결국 멜론을 계속 구매해야 음악을 들을 수 있는 멜론 구매용 미끼상품.
대박 신기 ! 혼자 사는 사람에게 딱 좋아요 :D 저는 집에 밤에는 무드등으로 써요. 진짜 대박 좋아요~~~ "아리아, 무드등 켜줘, 꺼줘" 불키고 끄려고 일어났다 누웠다 할 필요가 없음! 수면예약해놓고 잔잔한 음악 틀어놓으면 혼자여도 전혀 외롭지 않음 !!! 화장하고 출근할때는진짜두말하면 잔소리 !!! 노래도 들으면서 화장하고, 날씨 물어보고 옷 코디하고, 몇시인지 물어가며 준비하는데 완전 좋아요 ! 이제 휴대폰에 시계보려고 화장품 묻히는거 노노 !! 이제는 피자 치킨 배달까지 된다니! 완전 기대 *_*!!! (내일 시켜먹어볼 예정 ㅋㅋㅋ) 정말로 적극추천하고 싶은 여자싱글 필

In [106]:
for num, i in enumerate(pred_labels):
    if i == -1:
        print(review_texts[num])

귀도 어둡고, 머리도 나쁘고, 그저 노래만 부를 줄 아는군
솔직한 사용기 입니다. 일단 음성인식 블루투스 스피커 + 무드등을하나 산다고 해도 이가격이면 괜찮다고 생각합니다. 음성 인식률이나 다양한 정보는 아직 한참 미흡합니다. 정해진 규칙에 어느정도 맞춰서 대화해야하고 그외에는 다 알아듣지 못한다고 보면 됩니다. 음악하나는 잘 틀어줍니다. 그리고 이번에 업데이트되서 무드등기능이 좀더 업그레이드 되어 보기 좋더군요. 앞으로 소스도 공개하고 응용범위도 확대해 나간다니 기대해볼랍니다. SK니깐 손놓고 있진 않겠지요. 아마존의 에코, 구글의 구글홈 이제품들과 비교해서 일단 메리트는 한국말 대답이자연스럽고 멜론과 연동되는 정도 그리고 음질은 안꿀린다...이정도 인것 같습니다..아직은.... 초등학교 들어가기전 애들 언어교육할 상대 정도만 돼도 참 좋겠더군요. 시리정도 기대하시면 부족합니다...대신 목소리는 더좋아요..자연스럽고 이상 간단 솔직구매후기 였습니다.
현재까지 기능만보면 멜론가입안하면 전혀 쓸모없는 기계(인공지능이란 광고가 사기)입니다. 만약 3개월내(멜론쿠폰 3개월) 지식검색이나 뉴스 검색기능이 안되면 구매 의미가 없습니다. 빠른 조치 부탁드립니다.
와이파이 인식이 5분정도 걸려 그시간에 나자신을 되돌아보는 시간이되었고누구가 말을 잘 못알아들어 어머니가 자식한글 알려주는 마음을 들게되었지만 유일하게 화나는점은 알람 잘못끔ㅋ..
용한 경험에 따른 후기 자세히 기재합니다. 판매자보다는 개발,운영사에서 더 많은 노력이 필요하리라 생각되는 부분이 많습니다. [언어] 한국 최초의 음성인식 비서 서비스로는 만족합니다. (상업적으로 부족한 부분도 많지만.^^) 현재 한국어만 인식되고 있고 영어는 지원되지 않습니다. [인식률] 설명서에는 전원이 들어온 후에 약 1분 정도 인식이 안된다고 기재되어 있으나 실재 사용해보면 약 3~4분정도 인식률이 급격이 떨어집니다. 아마도 저의 네트워크 문제라기 보다는 누구 서버의 회선 또는 내부 설정이 충분하지 않은 듯합니다. 대략 5분 정

### sentiment dictionary 이용하여 나누기

In [46]:
tf_vectorizer, tf_features = tf_extractor(texts)
vocablist = [word for word, _ in sorted(tf_vectorizer.vocabulary_.items(), key=lambda x:x[1])]

In [47]:
# 각 feature 들의 감성 점수 list (감성사전에 없는 feature = 0)
sent_score = []
for i in vocablist:
    if i in sent_dict:
        sent_score.append(sent_dict[i])
    else:
        sent_score.append(0)

In [48]:
# 각 문헌의 점수를 계산
tf_score = []
for i in tf_features:
    tf_score.append(i.dot(sent_score)[0])
tf_score

[16.503492987043913,
 -5.997952963612364,
 0.5634197401186769,
 -4.0075183326530865,
 21.883267509417756,
 -0.7450948700129161,
 3.370555175226662,
 -29.432248760299643,
 5.106500916442445,
 12.085252988401644,
 -12.757437910475323,
 -0.6328930921061193,
 2.490329224779809,
 -0.9271787146144134,
 -15.104426972324632,
 7.373917864793675,
 -11.69013738178129,
 -5.330093455071581,
 4.996017020404211,
 -6.226069167343789,
 3.5322651388350064,
 7.138603539104491,
 2.8386760214406537,
 -5.928354130525635,
 -16.696745734028614,
 -4.17613264858293,
 -1.7600481357720967,
 8.358776156142511,
 -4.914627581857023,
 -2.7850863004763236,
 -32.363544268906445,
 3.0966924964507285,
 -11.534721979692113,
 22.72634797982358,
 -10.092595918498818,
 3.253946689379105,
 -3.460767764880818,
 -4.29908554726237,
 -5.848964831508074,
 3.3457090624423214,
 -3.943167797542783,
 -3.4646856458959547,
 -1.717739368404886,
 4.517915351777255,
 -0.5638340801130165,
 23.2635015022667,
 -2.06691098532715,
 12.678208926

In [49]:
pos = []
neg = []
neu = []
for num, score in enumerate(tf_score):
    if score > 0:
        pos.append(texts[num])
    elif score < 0:
        neg.append(texts[num])
    else:
        neu.append(texts[num])

### Dead kernel... 일단 저장..

In [17]:
with open('pos_texts.txt', 'w', encoding='utf-8') as p:
    for i in pos:
        p.write(i+'\n')
p.close()
len(pos)

1848

In [49]:
with open('neg_texts.txt', 'w', encoding='utf-8') as n:
    for i in neg:
        n.write(i+'\n')
n.close()
len(neg)

1555

In [50]:
with open('neu_texts.txt', 'w', encoding='utf-8') as ne:
    for i in neu:
        ne.write(i+'\n')
ne.close()
len(neu)

2

In [38]:
pos = []
with open('pos_texts.txt', 'r', encoding='utf-8') as p:
    for i in p.readlines():
        pos.append(i.split('\n')[0])
len(pos)

1848

In [39]:
pos[:5]

['침대 옆 공간은 좁지만 놓아야 할 것들이 꽤 많아서 늘 비좁은데, 침대맡에 하나 두고 저녁 생활을 도울 괜찮은 기능의 블루투스 스피커 + 라디오 알람시계를 소개합니다.  블루투스 스피커 / LED 알람시계 / 라디오 / 충전용USB 아이러브(iLuv) 모닝콜 2, 이 제품은 책상이나 주방, 특히 침대맡에 두기 딱 좋은 다기능 라디오 알람시계입니다. 딱히 라디오라고 정의하기는 애매한데 생긴 것이 라디오 같으니까...ㅎㅎ 크기는 사진에서 보이는 것보다 좀 작습니다.  한 뼘 조금 안되는 아담 사이즈. 침대 옆 탁자나 싱크대, 혹은 책상 앞에 올려 놓기 좋은 크기예요.  밤에 조명을 끈 방에서는 LED시계가 필요하죠. 그런데 시계의 화면이 엄청 큰!!!아주 잘 보여요. ^^[ LED 밝기조절 ]작은 불빛에도 수면에 방해 받는 분들이 있더라고요. 제 아내처럼. 그런 분들은 LED 밝기를 5단계로 조절하시면 됩니다. (스누즈 버튼) ▼ \xa0● Radio : 정해진 시간에 라디오가 켜집니다\xa0● Buzzer : 설정시간에 뚜뚜 부저음이 울려요아침에 두 번의 알람을 설정해 울릴 수 있고 두 가지 종류 중 선택합니다.  아침에는 두 개를 시간차 설정해서 확실히 깨면 되겠죠? 라디오로 깰 분위기 잡고 부저 알람으로 발딱 일어나기. (혹은 거꾸로...) ㅎㅎ 요게 라디오 알람시계를 찾는 이유 아니겠어요?※ 스누즈 버튼을 누르면 약 9분 후 알람이 다시 울립니다 주파수를 탐색해 듣거나 6개 채널 주파수가 저장이 됩니다. Sleep버튼으로 일정 시간 후 자동으로 꺼지도록 설정할 수 있어요. (15, 30, 45, 60, 90분) ※ 안테나는 긴 "선"이 뒤에 달려 있습니다. 펼쳐서 벽에 붙이거나 적절히 위치를 조절하면 신호가 잘 잡혀요.  스마트폰과 블루투스 연결로 MP3나 애플뮤직, 벅스 팟캐스트 같은 서비스의 스트리밍 음악을 들을 수 있어요. 해외 영어방송 채널이나 외국어 회화 들으시는 분들도 있으시죠.  [블루투스 페어링] 밤에 스마트폰 충전은 필수인데 USB충전단

In [40]:
neg = []
with open('neg_texts.txt', 'r', encoding='utf-8') as n:
    for i in n.readlines():
        neg.append(i.split('\n')[0])
len(neg)

1555

In [41]:
neg[:5]

['WAVE 를 입수한 김에 큰 맘먹고 리뷰를 한 번 써보려다가 결국 근사한 리뷰작성 근처에도 가지 못하고 사용준비 하는 과정에서 사소한 몇 가지 걸림돌 때문에 QA 항목 몇 가지 발견하고 중도 포기.... wave는 기가속도(olleh_GiGA_WiFi_4B16)는 몇 번을 시도해도 설정되지 않아서 결국 일반속도 와이파이(olleh_WiFi_4B16)으로 무선랜 설정을 마칠 수 있었다. 간신히 와이파이 설정 단계를 넘어 간 이후 AI 스피커 활용사례로 제일 많이 꼽히는 음악 감상을 위해 박스에 동봉된 네이버뮤직 선물쿠폰을 입력하다가 또 벽에 걸리고 말았다.이 단계에서는 갑자기 WAVE가 아닌 Naver Music 앱 QA로 전환된다~쿠폰에 적혀 있는 간단한 설명 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 아이폰 네이버뮤직 앱에서는 "이용권 구매 / 내 이용권" 메뉴 어디에서도 쿠폰 등록 버튼은 찾을 수 없다... 아이폰 네이버뮤직 앱에서 쿠폰 등록 메뉴 찾기에 실패하고 보유 중인 안드로이드폰 네이버뮤직 앱에 들어가서 찾아보니....쿠폰에 적혀 있는 " 모바일: 네이버뮤직앱 또는 웹 > 메뉴 > 이용권 > 쿠폰등록" 와는 조금 다른 단계로 찾긴 찾을 수 있었다."MY > 이용권 구매 > 내 이용권 정보" 페이지의 맨 하단에 있다....ㅠㅠ정말 블로그에 글을 적고 있는 이 순간까지도 설마 아이폰 앱이라고 해서 쿠폰등록 메뉴가 없을 리가..싶어서 다시 한 번 확인해봤다. 어딘가에 있는데 내가 황당한 착각을 하고 있는 것은 아닐 지?\xa0하지만 열번을 양보해서 어딘가에 쿠폰등록메뉴가 있긴 있더라도 적어도 쿠폰 안내문은 좀 더 정확하게 표현해줘야 하는 것 아닐까 싶다...',
 '최근 등장한 새로운 용어바로 스마트 스피커입니다.간단히 설명하면 음성인식이 가능해서 인터넷의 인공지능 서버를 통해 사용자의 요구에 대해 적절하게 반응해주는 장치입니다.하지만 가장 먼저는 음악 재생들의 기본적인 기능과 함께 함께 연결되어 있는 홈 네크워크를 적절하게 

In [4]:
neu = []
with open('neu_texts.txt', 'r', encoding='utf-8') as ne:
    for i in ne.readlines():
        neu.append(i.split('\n')[0])
len(neu)

2

### positive document

In [42]:
from tqdm import tqdm_notebook

In [43]:
# kornoun extractor로 명사 추출 for 미등록 단어
from kornounextractor.noun_extractor import extract
extract_texts = list(map(extract, lr_pos))
extract_texts = sum(extract_texts, [])

In [6]:
with open("./dic_pos.txt", 'w', encoding='UTF-8') as f:
    for i in extract_texts:
        f.write(i+'\tNNG\n')

In [ ]:
# 미등록단어 지정하여 Komoran 형태소 분석 
import konlpy.tag
dicpath = './dic_pos.txt'
komoran = konlpy.tag.Komoran(userdic= dicpath)
komo_nouns = list(map(komoran.nouns, pos))

In [ ]:
def remove_one_length(x):
    return len(x)>1

filtered_nouns = []
for noun in komo_nouns:
    filtered_noun = list(filter(remove_one_length, noun))
    filtered_nouns.append(filtered_noun)

In [ ]:
# 불용어 제거
with open(r'C:\Users\sormd\Desktop\수업_1학기\기계학습\과제\stopwordsKor.txt', 'r', encoding='utf8') as f:
    stopwords = f.read()

def remove_stopwords(x):
    for word in x:
        if word in stopwords:
            x.remove(word)
    return(x)

cleaned_nouns = list(map(remove_stopwords, filtered_nouns))

In [54]:
import collections
sum_nouns = sum(cleaned_nouns,[])
count_nouns = collections.Counter(sum_nouns)

In [55]:
collections.OrderedDict(count_nouns.most_common())

OrderedDict([('클로바', 5347),
             ('스피커', 4878),
             ('네이버', 3780),
             ('사용', 3093),
             ('인공지능', 2760),
             ('가능', 2667),
             ('기능', 2386),
             ('서비스', 1870),
             ('AI', 1684),
             ('프렌즈', 1586),
             ('음악', 1567),
             ('제품', 1453),
             ('연결', 1445),
             ('보다', 1384),
             ('함께', 1367),
             ('IoT', 1178),
             ('부터', 1160),
             ('정보', 1115),
             ('설정', 1108),
             ('이용', 1089),
             ('확인', 1026),
             ('활용', 983),
             ('기술', 978),
             ('LG', 956),
             ('필요', 934),
             ('스마트', 930),
             ('편리', 925),
             ('음성', 916),
             ('오늘', 915),
             ('노래', 913),
             ('영어', 908),
             ('아이들', 848),
             ('브라운', 844),
             ('검색', 838),
             ('블루투스', 832),
             ('버튼', 830),
             ('콘텐츠', 823),
   

In [56]:
sorted(count_nouns.items(), key=lambda i: i[1])

[('충전용USB', 1),
 ('LED시계', 1),
 ('Radio', 1),
 ('Buzzer', 1),
 ('설정시간', 1),
 ('부저음', 1),
 ('시간차', 1),
 ('부저', 1),
 ('Sleep버튼', 1),
 ('영어방송', 1),
 ('충전상태', 1),
 ('연애통화', 1),
 ('꿀기능', 1),
 ('AAA배터리', 1),
 ('설정사항', 1),
 ('용도예요', 1),
 ('검색유입', 1),
 ('와글와글', 1),
 ('음악취향', 1),
 ('스푼', 1),
 ('spoon', 1),
 ('라디오판', 1),
 ('클립네이버', 1),
 ('음성판', 1),
 ('팟빵팟캐스트', 1),
 ('Box3', 1),
 ('Vox음악', 1),
 ('사운드클라우드', 1),
 ('Mixerbox', 1),
 ('evermusic', 1),
 ('pro클라우드', 1),
 ('mp3몽땅', 1),
 ('cnspower', 1),
 ('은행업무', 1),
 ('입출금', 1),
 ('외부약속', 1),
 ('출타', 1),
 ('텔레비시청', 1),
 ('온라인고스톱', 1),
 ('베타버젼', 1),
 ('인식자체', 1),
 ('구글음성', 1),
 ('배우이름', 1),
 ('검색하기위', 1),
 ('사드린건데', 1),
 ('거주환경', 1),
 ('백 투 더 퓨처', 1),
 ('로버트', 1),
 ('저메키스', 1),
 ('바이센테니얼맨', 1),
 ('콜럼버스', 1),
 ('면면', 1),
 ('컬처', 1),
 ('CUMI', 1),
 ('포켓린트', 1),
 ('국제가전박람회', 1),
 ('1924년', 1),
 ('산업전시', 1),
 ('가전전시회', 1),
 ('smartphone4europe', 1),
 ('scoop', 1),
 ('자연언어', 1),
 ('Thinkstock', 1),
 ('shutterstock', 1),
 ('음향학', 1),
 ('지멘스', 1),
 ('보쉬', 1),
 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 그래프에 retina display 적용
#%config InlineBackend.figure_format = 'retina'

# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

In [ ]:
wordcloud = WordCloud(
    font_path = fontpath,
    width = 800,
    height = 600,
    #stopwords = lines,
    background_color="white"
)

wordcloud = wordcloud.generate_from_frequencies(word_freq)

plt.figure(figsize=(15, 11))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()